In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from huggingface_hub import HfApi, login
from torch.utils.data import Dataset
from datasets import load_dataset
import torch
import json
import torch

In [2]:
# Load the pre-trained BART model and tokenizer
model_name = 'facebook/bart-large'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Define a function to generate responses
def respond(input_text=""):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    response_ids = model.generate(input_ids, max_length=125, num_beams=5, early_stopping=True)
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response

# Test the function
output = respond(input_text="What is biology")
print(output)


with open("/kaggle/input/biologyrunto-dataset/chatbot_dataset.json", "r") as file:
    dataset=json.load(file)

def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding='max_length', truncation=True, max_length=128)
    outputs = tokenizer(examples['output'], padding='max_length', truncation=True, max_length=128)
    inputs['labels'] = outputs['input_ids']
    return inputs

tokenized_dataset = [tokenize_function(example) for example in dataset]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


What is it?


In [3]:
# Convert to a format compatible with the Trainer
class CustomDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.dataset = tokenized_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.dataset[idx].items()}

train_dataset = CustomDataset(tokenized_dataset)

# Define training arguments with a different run name
training_args = TrainingArguments(
    output_dir='./BiologyRunto-model',
    #run_name='BiologyRunto_Training_Run',  # Set a different run name
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)
 
# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Your dataset here
)

# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.846500
1000,0.953900
1500,0.726400
2000,0.601100
2500,0.486600


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2730, training_loss=0.8857908227941492, metrics={'train_runtime': 596.844, 'train_samples_per_second': 9.143, 'train_steps_per_second': 4.574, 'total_flos': 1478236226715648.0, 'train_loss': 0.8857908227941492, 'epoch': 3.0})

In [5]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('/kaggle/working/BiologyRunto-model')
tokenizer.save_pretrained('/kaggle/working/BiologyRunto-model')


('/kaggle/working/BiologyRunto-model/tokenizer_config.json',
 '/kaggle/working/BiologyRunto-model/special_tokens_map.json',
 '/kaggle/working/BiologyRunto-model/vocab.json',
 '/kaggle/working/BiologyRunto-model/merges.txt',
 '/kaggle/working/BiologyRunto-model/added_tokens.json')

In [7]:
# Load the model and tokenizer
model_name = '/kaggle/working/BiologyRunto-model'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [14]:
def respond(input_text=""):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    response_ids = model.generate(input_ids, max_length=130, num_beams=10, early_stopping=True)
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response

In [15]:
output=respond(input_text="what are the components of a plant cell")
print(output)

The components of a plant cell include:
1. Cytoplasm: The gel-like substance that fills the cell.
2. Nucleus: The organelle that contains the cell's genetic material (DNA) and controls cellular activities.
3. Endoplasmic reticulum: The membrane-bound organelles that store and transport nutrients, waste products, and waste products.
4. Golgi apparatus: The apparatus that maintains cell shape and maintains turgor pressure.


In [16]:
login(token="put your assess token here")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [17]:
api = HfApi()
api.upload_folder(
    folder_path="/kaggle/working/BiologyRunto-model",
    repo_id="AdesegunDaniel/BiologyRunto",
    repo_type="model"
)

optimizer.pt:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

events.out.tfevents.1735561510.c0d6bc280cc8.30.0:   0%|          | 0.00/7.38k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AdesegunDaniel/BiologyRunto/commit/2fe1fcbd72d1f55334ac18eed9888878ec6f483a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2fe1fcbd72d1f55334ac18eed9888878ec6f483a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AdesegunDaniel/BiologyRunto', endpoint='https://huggingface.co', repo_type='model', repo_id='AdesegunDaniel/BiologyRunto'), pr_revision=None, pr_num=None)

In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
import os


model_name = "AdesegunDaniel/BiologyRunto"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

def respond(input_text=""):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    response_ids = model.generate(input_ids, max_length=100, num_beams=3, early_stopping=True)
    response = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response


config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [2]:
user_input = 'what is skeleton'
bot_response = respond(user_input)
print(bot_response)

Skeleton is a structure made of bone and cartilage that provides support and protection to the body's organs and tissues. It is composed of microtubules arranged in a '9+2' pattern and provides structural support and rigidity. Skeletons are found in vertebrates, including humans, and play a crucial role in maintaining overall health and function.
